In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings()
database_loc = ("./chroma_db_test1")

vectorstore = Chroma(persist_directory=database_loc,
                     embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 8})


In [ ]:
from langchain_ollama import OllamaLLM
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OllamaLLM(model='llama3.2:latest')
from langchain_core.prompts import PromptTemplate

template = """You are an AI assistant for the Vesuvius Challenge discord, which is a a global initiative using machine learning and computer vision to decipher the contents of the charred Herculaneum scrolls, aiming to unlock ancient knowledge without physically damaging the fragile artifacts. Additionally you may be asked questions about the overall sentiment of the discord chats or high level topics from the discord server. Answer the question using the provided information, which are chats from the discord server, which may contain questions, replies, etc. Stay focused only the question that follows "Question:" and give as much detail as possible.
{context}
Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}



In [ ]:
from langgraph.graph import START, StateGraph

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))
response = graph.invoke({"question": "Who is the expert on ink algorithms?"})

print(f'Answer: {response["answer"]}\n\n')
print("*" * 80)
print(f'Context: {response["context"]}\n\n')